In [10]:
import torch
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T
import torch.nn as nn
import nemo.collections.asr as nemo_asr
from typing import Optional


In [2]:
from nemo.collections.tts.models import FastPitchModel
spec_generator = FastPitchModel.from_pretrained("tts_en_fastpitch")

[NeMo W 2024-10-17 09:13:35 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/nemo/collections/tts/modules/common.py:206: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
      @amp.autocast(False)
    


[NeMo I 2024-10-17 09:13:36 cloud:58] Found existing object /home/keagan/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.
[NeMo I 2024-10-17 09:13:36 cloud:64] Re-using file from: /home/keagan/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo
[NeMo I 2024-10-17 09:13:36 common:924] Instantiating model from pre-trained checkpoint


 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
[NeMo W 2024-10-17 09:13:48 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2024-10-17 09:13:48 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
 

[NeMo I 2024-10-17 09:13:48 features:289] PADDING: 1


[NeMo W 2024-10-17 09:13:48 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/nemo/core/connectors/save_restore_connector.py:571: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related t

[NeMo I 2024-10-17 09:13:49 save_restore_connector:249] Model FastPitchModel was successfully restored from /home/keagan/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_align/b7d086a07b5126c12d5077d9a641a38c/tts_en_fastpitch_align.nemo.


In [3]:
import soundfile as sf
parsed = spec_generator.parse("You can type your sentence here to get nemo to produce speech.This is a test to see how well fast pitch is")
spectrogram = spec_generator.generate_spectrogram(tokens=parsed)

[NeMo W 2024-10-15 10:54:09 fastpitch:291] parse() is meant to be called in eval mode.
 NeMo-text-processing :: INFO     :: Skipping post-processing of You can type your sentence here to get nemo to produce speech dot This is a test to see how well fast pitch is for '.'
[NeMo W 2024-10-15 10:54:09 fastpitch:368] generate_spectrogram() is meant to be called in eval mode.


In [4]:
spectrogram.shape

torch.Size([1, 80, 642])

In [3]:
Fe_encoder = spec_generator.fastpitch.encoder
Fd_decoder = spec_generator.fastpitch.decoder

In [4]:
Fe_encoder

FFTransformerEncoder(
  (pos_emb): PositionalEmbedding()
  (drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (dec_attn): MultiHeadAttn(
        (qkv_net): Linear(in_features=384, out_features=192, bias=True)
        (drop): Dropout(p=0.1, inplace=False)
        (dropatt): Dropout(p=0.1, inplace=False)
        (o_net): Linear(in_features=64, out_features=384, bias=False)
        (layer_norm): ConditionalLayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
      (pos_ff): PositionwiseConvFF(
        (CoreNet): Sequential(
          (0): Conv1d(384, 1536, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): ReLU()
          (2): Conv1d(1536, 384, kernel_size=(3,), stride=(1,), padding=(1,))
          (3): Dropout(p=0.1, inplace=False)
        )
        (layer_norm): ConditionalLayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (cond_input): ConditionalInput()
  (word_emb): Embedding(115, 384, padding

In [6]:
class ConformerEncoder256(nn.Module):
    def __init__(self, original_encoder):
        super(ConformerEncoder256, self).__init__()
        self.encoder = original_encoder
        # Add a 1D convolution with stride to downsample to 256
        self.downsample = nn.Conv1d(in_channels=512, out_channels=256, kernel_size=1, stride=2) # Example

    def forward(self, audio_signal, length):
        # Pass through original encoder
        embeddings, lengths = self.encoder(audio_signal=audio_signal, length=length)
        # Reduce the embedding size
        embeddings = self.downsample(embeddings)
        return embeddings, lengths

In [7]:
def cosine_similarity(v1, v2):
    return F.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0), dim=-1).item()

def group_similar_vectors(vectors, threshold, vector_duration):
    grouped_vectors = []
    durations = []

    current_group = [vectors[:, 0]]  # Start with the first vector (slice across batch dimension)
    current_duration = vector_duration

    for i in range(1, vectors.shape[-1]):
        sim = cosine_similarity(vectors[:, i], vectors[:, i - 1])
        # print(sim)
        # If cosine similarity is above threshold, group the vectors
        if sim > threshold:
            # print("Exceeded Threshold")
            current_group.append(vectors[:, i])
            current_duration += vector_duration
        else:
            # Compute the average of the current group and save it
            averaged_vector = torch.mean(torch.stack(current_group, dim=-1), dim=-1)
            grouped_vectors.append(averaged_vector)
            durations.append(current_duration)

            # Start a new group
            current_group = [vectors[:, i]]
            current_duration = vector_duration

    # Append the last group
    if current_group:
        averaged_vector = torch.mean(torch.stack(current_group, dim=-1), dim=-1)
        grouped_vectors.append(averaged_vector)
        durations.append(current_duration)

    return torch.stack(grouped_vectors, dim=-1), durations

# Example usage
# Assume `content_embeddings` is the input tensor of shape [1, 256, num_content_vectors]
# `T`: Cosine similarity threshold
# `vector_duration`: The duration of a single vector (calculated as ~46.44 milliseconds)

def duration_augmented_representation(content_embeddings, T=0.9, vector_duration=46.44):
    # Remove the batch dimension for processing
    content_vectors = content_embeddings.squeeze(0)  # Shape: [256, num_content_vectors]

    grouped_vectors, new_durations = group_similar_vectors(content_vectors, T, vector_duration)

    # Convert durations to seconds if needed
    new_durations_in_seconds = [d / 1000 for d in new_durations]

    # Add the batch dimension back to the output
    grouped_vectors = grouped_vectors.unsqueeze(0)  # Shape: [1, 256, num_grouped_vectors]

    return grouped_vectors, new_durations_in_seconds

In [8]:
audio_path = '/home/keagan/Documents/projects/SelfVC/data/audios/14_208_000001_000000.wav'
conformer_encoder_256 = torch.load('/home/keagan/Documents/projects/SelfVC/models/conformer_encoder_v2.pth')
mel_transform = T.MelSpectrogram(
            sample_rate=22050,
            n_mels=80,
            n_fft=1024,
            hop_length=256,
            win_length=1024,
        )
waveform, sr = torchaudio.load(audio_path)
mel_spectrogram = mel_transform(waveform)

[NeMo W 2024-10-15 10:54:40 nemo_logging:349] /tmp/ipykernel_103650/1129351816.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
      conformer_encoder_256 = torch.load('/h

In [9]:

mel_spectrogram.shape

torch.Size([1, 80, 846])

In [10]:
# output_vectors = torch.randn(1,256,100)  # Replace with actual model output

T = 0.925  # Cosine similarity threshold
grouped_vectors, new_durations = duration_augmented_representation(mel_spectrogram, T)


In [11]:
grouped_vectors.shape

torch.Size([1, 80, 350])

In [12]:
len(new_durations)

350

In [15]:
lengths = torch.full((mel_spectrogram.shape[0],), mel_spectrogram.shape[2], dtype=torch.int32)
# print(lengths.shape)
# with autocast():
embeddings, embeddings_length = conformer_encoder_256(
    audio_signal=mel_spectrogram.cuda(),
    length=lengths.cuda()
)

In [16]:
embeddings.shape

torch.Size([1, 256, 106])

In [17]:
grouped_vectors, new_durations = duration_augmented_representation(embeddings, 0.9)

In [18]:
grouped_vectors.shape

torch.Size([1, 256, 1])

In [19]:
new_durations

[4.922639999999997]

In [79]:

import torch

# Example input: token IDs (Batch, Sequence Length)(Phoneme SEquence)
input_tokens = torch.randint(0, 100, (1,256)).cuda()

# Pass through FastPitch encoder
encoder_outputs = Fe_encoder(input_tokens)

In [24]:
input_tokens.shape

torch.Size([1, 256])

In [80]:
encoder_outputs[0].shape

torch.Size([1, 256, 384])

In [78]:
# Fe_encoder = Fe_encoder.cuda()
import torch.nn as nn

global_avg_pooling = nn.AdaptiveAvgPool1d(384)  # Set the output size to 384

output_vectors = torch.randn(1,256,200)
# .cuda()  # Replace with actual model output
x_pooled = global_avg_pooling(output_vectors)
# input_tensor = x_pooled.to(torch.int)
Fe_encoder(input=x_pooled)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [28]:
output_vectors

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [4]:

Fe_encoder


FFTransformerEncoder(
  (pos_emb): PositionalEmbedding()
  (drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (dec_attn): MultiHeadAttn(
        (qkv_net): Linear(in_features=384, out_features=192, bias=True)
        (drop): Dropout(p=0.1, inplace=False)
        (dropatt): Dropout(p=0.1, inplace=False)
        (o_net): Linear(in_features=64, out_features=384, bias=False)
        (layer_norm): ConditionalLayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
      (pos_ff): PositionwiseConvFF(
        (CoreNet): Sequential(
          (0): Conv1d(384, 1536, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): ReLU()
          (2): Conv1d(1536, 384, kernel_size=(3,), stride=(1,), padding=(1,))
          (3): Dropout(p=0.1, inplace=False)
        )
        (layer_norm): ConditionalLayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (cond_input): ConditionalInput()
  (word_emb): Embedding(115, 384, padding

# Custom Fe Model

In [97]:
class PositionwiseConvFF(nn.Module):
    def __init__(self, d_model, d_inner, kernel_size, dropout, pre_lnorm=False):
        super(PositionwiseConvFF, self).__init__()

        self.d_model = d_model
        self.d_inner = d_inner
        self.dropout = dropout

        self.CoreNet = nn.Sequential(
            nn.Conv1d(d_model, d_inner, kernel_size, 1, (kernel_size // 2)),
            nn.ReLU(),
            # nn.Dropout(dropout),  # worse convergence
            nn.Conv1d(d_inner, d_model, kernel_size, 1, (kernel_size // 2)),
            nn.Dropout(dropout),
        )
        self.layer_norm = nn.LayerNorm(d_model)
        self.pre_lnorm = pre_lnorm

    def forward(self, inp):
        return self._forward(inp)

    def _forward(self, inp):
        if self.pre_lnorm:
            # layer normalization + positionwise feed-forward
            core_out = inp.transpose(1, 2)
            core_out = self.CoreNet(self.layer_norm(core_out).to(inp.dtype))
            core_out = core_out.transpose(1, 2)

            # residual connection
            output = core_out + inp
        else:
            # positionwise feed-forward
            core_out = inp.transpose(1, 2)
            core_out = self.CoreNet(core_out)
            core_out = core_out.transpose(1, 2)

            # residual connection + layer normalization
            output = self.layer_norm(inp + core_out).to(inp.dtype)

        return output


In [96]:
class MultiHeadAttn(nn.Module):
    def __init__(self, n_head, d_model, d_head, dropout, dropatt=0.1,
                 pre_lnorm=False):
        super(MultiHeadAttn, self).__init__()

        self.n_head = n_head
        self.d_model = d_model
        self.d_head = d_head
        self.scale = 1 / (d_head ** 0.5)
        self.pre_lnorm = pre_lnorm

        self.qkv_net = nn.Linear(d_model, 3 * n_head * d_head)
        self.drop = nn.Dropout(dropout)
        self.dropatt = nn.Dropout(dropatt)
        self.o_net = nn.Linear(n_head * d_head, d_model, bias=False)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, inp, attn_mask=None):
        return self._forward(inp, attn_mask)

    def _forward(self, inp, attn_mask=None):
        residual = inp

        if self.pre_lnorm:
            # layer normalization
            inp = self.layer_norm(inp)

        n_head, d_head = self.n_head, self.d_head

        head_q, head_k, head_v = torch.chunk(self.qkv_net(inp), 3, dim=2)
        head_q = head_q.view(inp.size(0), inp.size(1), n_head, d_head)
        head_k = head_k.view(inp.size(0), inp.size(1), n_head, d_head)
        head_v = head_v.view(inp.size(0), inp.size(1), n_head, d_head)

        q = head_q.permute(2, 0, 1, 3).reshape(-1, inp.size(1), d_head)
        k = head_k.permute(2, 0, 1, 3).reshape(-1, inp.size(1), d_head)
        v = head_v.permute(2, 0, 1, 3).reshape(-1, inp.size(1), d_head)

        attn_score = torch.bmm(q, k.transpose(1, 2))
        attn_score.mul_(self.scale)

        if attn_mask is not None:
            attn_mask = attn_mask.unsqueeze(1).to(attn_score.dtype)
            attn_mask = attn_mask.repeat(n_head, attn_mask.size(2), 1)
            attn_score.masked_fill_(attn_mask.to(torch.bool), -float('inf'))

        attn_prob = F.softmax(attn_score, dim=2)
        attn_prob = self.dropatt(attn_prob)
        attn_vec = torch.bmm(attn_prob, v)

        attn_vec = attn_vec.view(n_head, inp.size(0), inp.size(1), d_head)
        attn_vec = attn_vec.permute(1, 2, 0, 3).contiguous().view(
            inp.size(0), inp.size(1), n_head * d_head)

        # linear projection
        attn_out = self.o_net(attn_vec)
        attn_out = self.drop(attn_out)

        if self.pre_lnorm:
            # residual connection
            output = residual + attn_out
        else:
            # residual connection + layer normalization
            output = self.layer_norm(residual + attn_out)

        output = output.to(attn_out.dtype)

        return output

In [128]:
class TransformerLayer(nn.Module):
    def __init__(self, n_head, d_model, d_head, d_inner, kernel_size, dropout,
                 **kwargs):
        super(TransformerLayer, self).__init__()

        self.dec_attn = MultiHeadAttn(n_head, d_model, d_head, dropout, **kwargs)
        self.pos_ff = PositionwiseConvFF(d_model, d_inner, kernel_size, dropout,
                                         pre_lnorm=kwargs.get('pre_lnorm'))

    def forward(self, dec_inp, mask=None):
        # output = self.dec_attn(dec_inp, attn_mask=~mask.squeeze(2))
        output = self.dec_attn(dec_inp)
        # output *= mask
        output = self.pos_ff(output)
        # output *= mask
        return output

In [98]:
def mask_from_lens(lens, max_len: Optional[int] = None):
    if max_len is None:
        max_len = lens.max()
    ids = torch.arange(0, max_len, device=lens.device, dtype=lens.dtype)
    mask = torch.lt(ids, lens.unsqueeze(1))
    return mask

In [130]:
class FFTransformer(nn.Module):
    def __init__(self, n_layer, n_head, d_model, d_head, d_inner, kernel_size,
                 dropout, dropatt, dropemb=0.0, embed_input=True,
                 n_embed=None, d_embed=None, padding_idx=0, pre_lnorm=False):
        super(FFTransformer, self).__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.d_head = d_head
        self.padding_idx = padding_idx

        # if embed_input:
        #     self.word_emb = nn.Embedding(n_embed, d_embed or d_model,
        #                                  padding_idx=self.padding_idx)
        # else:
        #     self.word_emb = None

        # self.pos_emb = PositionalEmbedding(self.d_model)
        self.drop = nn.Dropout(dropemb)
        self.layers = nn.ModuleList()

        for _ in range(n_layer):
            self.layers.append(
                TransformerLayer(
                    n_head, d_model, d_head, d_inner, kernel_size, dropout,
                    dropatt=dropatt, pre_lnorm=pre_lnorm)
            )

    def forward(self, dec_inp, seq_lens=None, conditioning=0):
        # if self.word_emb is None:
        #     inp = dec_inp
        #     mask = mask_from_lens(seq_lens).unsqueeze(2)
        # else:
        #     inp = self.word_emb(dec_inp)
        #     # [bsz x L x 1]
        #     mask = (dec_inp != self.padding_idx).unsqueeze(2)

        # pos_seq = torch.arange(inp.size(1), device=inp.device).to(inp.dtype)
        # pos_emb = self.pos_emb(pos_seq) * mask

        # out = self.drop(inp + pos_emb + conditioning)
        # mask = mask_from_lens(seq_lens).unsqueeze(2)
        # mask = (dec_inp != self.padding_idx).unsqueeze(2)
        # print(mask.shape)
        out = self.drop(dec_inp + conditioning)

        for layer in self.layers:
            # out = layer(out, mask=mask)
            out = layer(out)

        # out = self.drop(out)
        return out

In [131]:
custom_encoder = FFTransformer(
    n_layer=6, n_head=1,
    d_model=512,
    d_head=64,
    d_inner=1536,
    kernel_size=3,
    dropout=0.1,
    dropatt=0.1,
    dropemb=0.0,
    embed_input=True,
    d_embed=384,
    n_embed=None,
    padding_idx=0)

In [120]:
custom_encoder

FFTransformer(
  (drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x TransformerLayer(
      (dec_attn): MultiHeadAttn(
        (qkv_net): Linear(in_features=512, out_features=192, bias=True)
        (drop): Dropout(p=0.1, inplace=False)
        (dropatt): Dropout(p=0.1, inplace=False)
        (o_net): Linear(in_features=64, out_features=512, bias=False)
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (pos_ff): PositionwiseConvFF(
        (CoreNet): Sequential(
          (0): Conv1d(512, 1536, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): ReLU()
          (2): Conv1d(1536, 512, kernel_size=(3,), stride=(1,), padding=(1,))
          (3): Dropout(p=0.1, inplace=False)
        )
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
)

In [121]:
import torch.nn as nn

global_avg_pooling = nn.AdaptiveAvgPool1d(512)  # Set the output size to 384

output_vectors = torch.randn(1,256,200)
# .cuda()  # Replace with actual model output
x_pooled = global_avg_pooling(output_vectors)
# input_tensor = x_pooled.to(torch.int)
# Fe_encoder(input=x_pooled)

In [122]:
x_pooled.shape

torch.Size([1, 256, 512])

In [132]:
output_vectors = torch.randn(1,256,512) #(batch,timesteps,embeddings)

In [133]:
test_enc_out = custom_encoder(output_vectors)

In [134]:
test_enc_out.shape

torch.Size([1, 256, 512])

In [86]:
test_dec_out = custom_encoder(test_enc_out)

In [85]:
test_dec_out.shape

torch.Size([1, 200, 512])

# PitchPredictor

In [89]:
pitch_predictor = spec_generator.fastpitch.pitch_predictor

In [91]:
pitch_predictor

TemporalPredictor(
  (cond_input): ConditionalInput()
  (layers): ModuleList(
    (0): ConvReLUNorm(
      (conv): Conv1d(384, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (norm): ConditionalLayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): ConvReLUNorm(
      (conv): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (norm): ConditionalLayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

# Custom PitchPredictor


In [92]:
class ConvReLUNorm(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=1, dropout=0.0):
        super(ConvReLUNorm, self).__init__()
        self.conv = torch.nn.Conv1d(in_channels, out_channels,
                                    kernel_size=kernel_size,
                                    padding=(kernel_size // 2))
        self.norm = torch.nn.LayerNorm(out_channels)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, signal):
        out = F.relu(self.conv(signal))
        out = self.norm(out.transpose(1, 2)).transpose(1, 2).to(signal.dtype)
        return self.dropout(out)

In [139]:
class TemporalPredictor(nn.Module):
    """Predicts a single float per each temporal location"""

    def __init__(self, input_size, filter_size, kernel_size, dropout,
                 n_layers=2, n_predictions=1):
        super(TemporalPredictor, self).__init__()

        self.layers = nn.Sequential(*[
            ConvReLUNorm(input_size if i == 0 else filter_size, filter_size,
                         kernel_size=kernel_size, dropout=dropout)
            for i in range(n_layers)]
        )
        self.n_predictions = n_predictions
        self.fc = nn.Linear(filter_size, self.n_predictions, bias=True)

    def forward(self, enc_out):
        # enc_out_mask
        out = enc_out 
        # * enc_out_mask
        out = self.layers(out.transpose(1, 2)).transpose(1, 2)
        out = self.fc(out) 
        # * enc_out_mask
        return out


In [140]:
pitch_predictor = TemporalPredictor(
        512,
        filter_size=256,
        kernel_size=3,
        dropout=0.1, n_layers=2,
        n_predictions=1
    )

In [141]:
pitch_predictor

TemporalPredictor(
  (layers): Sequential(
    (0): ConvReLUNorm(
      (conv): Conv1d(512, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): ConvReLUNorm(
      (conv): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [146]:
test_enc_out.shape

torch.Size([1, 256, 512])

In [147]:
output_vectors = torch.randn(1,100,512)

In [148]:
pitch_pred_out = pitch_predictor(output_vectors)

In [149]:
pitch_pred_out.shape

torch.Size([1, 100, 1])

In [151]:
import numpy as np
pitch_contour = np.random.rand(50) 

In [153]:
pitch_contour.shape

(50,)

# Duration Predictor

In [90]:
duration_predictor = spec_generator.fastpitch.duration_predictor